<a href="https://colab.research.google.com/github/maldil/NER/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using BERT on IOB tagging

In [2]:
!pip install transformers seqeval[gpu]

     |████████████████████████████████| 3.1 MB 4.0 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 3.3 MB 40.8 MB/s 
     |████████████████████████████████| 895 kB 52.7 MB/s 
     |████████████████████████████████| 596 kB 46.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=bddaa2f3f2e62166172deb662af63392e06d6574f9f3e45f4d7bd30bcef05e4b
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [33]:
data = open("S21-gene-train.txt").readlines()
line=""
tags=""
list_data = []
for x in data:
  if len(x)==1 and x== "\n":
    list_data.append({'sentence':line[1:],'word_labels':tags[1:]})
    line=""
    tags=""
  else:
    line=line+" "+x.strip().split('\t')[1]
    tags=tags+","+x.strip().split('\t')[2]
df = pd.DataFrame(list_data) 
df.drop_duplicates().reset_index(drop=True)
df.head(20)
# print(data[0])
# data = pd.read_csv("S21-gene-train.txt", sep='\t',)
# data.head(20)

,sentence,word_labels
0,Comparison with alkaline phosphatases and 5 - ...,"O,O,B,I,O,B,I,I,O"
1,Pharmacologic aspects of neonatal hyperbilirub...,"O,O,O,O,O,O"
2,When CSF [ HCO3 -] is shown as a function of C...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,Flurazepam thus appears to be an effective hyp...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
4,Beta blocking agents .,"O,O,O,O"
5,When extracorporeal CO2 removal approximated C...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
6,Intravenous administration ( 25 mg / kg ) of c...,"O,O,O,O,O,O,O,O,O,B,I,O,O,O,O,O,O,O,O,O,O,O,O,..."
7,Teratological study of etoperidone in the rat ...,"O,O,O,O,O,O,O,O,O,O"
8,Colorectal polyps should be removed endoscopic...,"O,O,O,O,O,O,O,O,O"
9,The variable HMG dosage regimen was found to o...,"O,O,B,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
